In [2]:
import pandas as pd
import cptac
import numpy as np
import requests
import json
import deva
import binarization_functions_1 as bf
#import scisurv

In [3]:
en = cptac.Endometrial()
clinical = en.get_clinical()
proteomics = en.get_proteomics()

In [22]:
follow_up = pd.read_excel('../Follow_Up_Data/UCEC_followup_9_12.xlsx')

In [23]:
#We are only looking at the discovery cohort, so we will separate the data into two tables based on cohort
cohort_filter = follow_up['Cohort'] == "Disc"
discovery_cohort = follow_up[cohort_filter]
other_cohort = follow_up[~cohort_filter]

In [24]:
unique_id_follow_up = list(set(discovery_cohort['Case ID']))
unique_id_clinical = list(set(clinical['Patient_ID']))

In [25]:
cols_to_drop = ['Tumor code', 'ECOG Score', 
                'Karnofsky Score', 'Perform Status Scale: Timing', 
                'Other Perform Scale Timing', 
                'Specify Other New Tumor Site', 'Tamoxifen']

discovery_cohort = discovery_cohort.drop(cols_to_drop, axis=1)

for col in discovery_cohort.columns:
    if len(discovery_cohort[col].value_counts()) <= 1:
        discovery_cohort = discovery_cohort.drop(col, axis=1)
        
things_to_replace = ['Not Reported/ Unknown', 'Reported/ Unknown', 
                     'Not Applicable', 'na', 'unknown', 'Not Performed', 
                     'Unknown tumor status', 'Unknown', 
                     'Unknown Tumor Status', 'Not specified']

for col in discovery_cohort.columns:
    discovery_cohort[col] = discovery_cohort[col].replace(things_to_replace, np.nan)
    if np.issubdtype(discovery_cohort[col].dtype, np.number):
        mean = discovery_cohort[col].mean()
        discovery_cohort[col]= bf.binarizeCutOff(discovery_cohort, col, mean,
                                                 "Above_Mean("+str(round(mean, 2))+")",
                                                 "Below_Mean("+str(round(mean, 2))+")")

In [35]:
discovery_cohort = discovery_cohort.rename({'Case ID': 'Patient_ID'}, axis='columns')

In [46]:
patient_data = pd.merge(clinical, discovery_cohort, on = 'Patient_ID')

In [50]:
patient_data

,Patient_ID,Proteomics_Tumor_Normal,Country,Histologic_Grade_FIGO,Myometrial_invasion_Specify,Histologic_type,Treatment_naive,Tumor_purity,Path_Stage_Primary_Tumor-pT,Path_Stage_Reg_Lymph_Nodes-pN,...,Success of Initial Treatment,Success of Treatment@ Followup,New Tumor After Initial TRT,Path Diag to new Tumor (days),Additional Surgery ?,Path Diag to Surgery (days),Residual Tumor After Surgery,Radiation for New Tumor,Pharmaceutical for New Tumor,Immunological for New Tumor
0,C3L-00006,Tumor,United States,FIGO grade 1,under 50 %,Endometrioid,YES,Normal,pT1a (FIGO IA),pN0,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C3L-00006,Tumor,United States,FIGO grade 1,under 50 %,Endometrioid,YES,Normal,pT1a (FIGO IA),pN0,...,NaN,Complete Remission,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C3L-00006,Tumor,United States,FIGO grade 1,under 50 %,Endometrioid,YES,Normal,pT1a (FIGO IA),pN0,...,NaN,Complete Remission,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C3L-00006,Adjacent_normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C3L-00006,Adjacent_normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Complete Remission,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,C3L-00006,Adjacent_normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Complete Remission,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,C3L-00008,Tumor,United States,FIGO grade 1,under 50 %,Endometrioid,YES,Normal,pT1a (FIGO IA),pNX,...,NaN,Complete Remission,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,C3L-00008,Tumor,United States,FIGO grade 1,under 50 %,Endometrioid,YES,Normal,pT1a (FIGO IA),pNX,...,NaN,Complete Remission,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,C3L-00008,Tumor,United States,FIGO grade 1,under 50 %,Endometrioid,YES,Normal,pT1a (FIGO IA),pNX,...,NaN,Complete Remission,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,C3L-00032,Tumor,United States,FIGO grade 2,under 50 %,Endometrioid,YES,Normal,pT1a (FIGO IA),pN0,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
